In [2]:
import math
import numpy as np
import seaborn as sns
from time import time
import pandas as pd
import pymcmcstat
import matplotlib.pyplot as plt

In [3]:
def model_magnitud(theta, data):
    mag_v_sun = -26.72

    phase_angle = data['solar_phase_angle']
    range_sat = data['range_sat']
    cross_section = theta['cross_section']
    albedo = theta['albedo'] 
    mix_coef = theta['mix_coef']

   # Change degrees to radian
    degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
    coe_1 = 2/(3*np.pi)
    coe_2 = 1/(4*np.pi)

    phase_angle_rad = degtorad(phase_angle)	

   
    part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
   
    mag = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coef * part1) + coe_2) + 5*np.log10(range_sat)
    return mag

In [72]:
 def residual_calc(theta, data_x, data_y):
    residual = data_y - model_magnitud(theta, data_x)
    return residual

### Read Data and create new dataframe

In [13]:
path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
# path = '/home/kero/Documents/PhD/re-analyse/test.csv'
data_sat = pd.read_csv(path)

data_in = pd.DataFrame()
data_in['solar_phase_angle'] = data_sat['obs_phase_angle']
data_in['range_sat'] = data_sat['range_sat']
data_in['mag_observation'] = data_sat['mag_observation']
# data_in['solar_phase_angle'] = data_sat['SunPhaseAng']
# data_in['range_sat'] = data_sat['ObsTrailLength']
# data_in['mag_observation'] = data_sat['ObsMag']
data_in['e_mag'] = data_sat['mag_observation_e']
n = data_in.shape[0]

In [8]:
mean = 0.8
std_dev = 0.35
num_reps = n
num_simulations = 1000

cross_section_random = np.random.normal(mean, std_dev, num_reps).round(2)

In [9]:
mean = 0.55
std_dev = 0.2
num_reps = n
num_simulations = 1000

albedo_random = np.random.normal(mean, std_dev, num_reps).round(2)

In [10]:
mean = 0.75
std_dev = 0.12
num_reps = n
num_simulations = 1000

mix_coef_random = np.random.normal(mean, std_dev, num_reps).round(2)

In [24]:
def sum_squares(theta, data):

    rest = (data['mag_observation'] + data['e_mag']) + model_magnitud(theta, data)
    residual = (rest ** 2).sum(axis=0)/(rest.shape[0] - theta.shape[1])
    estimator = [residual, rest]
    
    return estimator

In [25]:
for i in range(10):
    # Choose random inputs for the sales targets and percent to target
    mix_coef = np.random.choice(mix_coef_random)
    albedo = np.random.choice(albedo_random)
    cross_section = np.random.choice(cross_section_random)
    # print(mix_coef, albedo, cross_section)


    df = pd.DataFrame(index=range(num_reps), data={'mix_coef': mix_coef,
                                                    'albedo': albedo,
                                                    'cross_section' : cross_section})

    model_value = model_magnitud(df, data_in)

    estimador = sum_squares(df, data_in)
    while estimador[0] != 1.0:
        estimador = sum_squares(df, data_in)
        print(estimador[0])

    print(estimador[0])
    




1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704860186
1.9361663704

KeyboardInterrupt: 

In [ ]:
estimador[0] = 


while estimador[0] != 1.0:
    mix_coef = np.random.choice(mix_coef_random)
    albedo = np.random.choice(albedo_random)
    cross_section = np.random.choice(cross_section_random)
    # print(mix_coef, albedo, cross_section)


    df = pd.DataFrame(index=range(num_reps), data={'mix_coef': mix_coef,
                                                    'albedo': albedo,
                                                    'cross_section' : cross_section})

    model_value = model_magnitud(df, data_in)

    estimador = sum_squares(df, data_in)



In [15]:
print(estimador[0])
print(estimador[1])

1.630091045673975
0      4.142995
1     -1.168826
2     -1.353674
3     -0.104737
4     -0.026946
         ...   
139   -3.293141
140   -3.378317
141   -3.742328
142   -3.562931
143   -3.076756
Length: 144, dtype: float64
